If you want to train/test the model, run the following modules:
- Dependencies (ALL)
- CLIP FINETUNE: Prepare Data, Neural Network (ignore import data and generate embeddings)
  

# Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from tqdm.auto import tqdm
import kagglehub
import matplotlib.pyplot as plt
import os
import torch
import pandas as pd
from tqdm import tqdm
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torchvision
from torch.utils.data import Dataset, DataLoader
import math
from tqdm.auto import tqdm
import torch.nn as nn
from pathlib import Path


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Mounted at /content/drive
Using device: cuda


# CLIP FINETUNE

Start from prepare data, No need to run "Import data" and

## Import Data from Kaggle

In [ ]:
# Download latest version
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

print("Path to dataset files:", path)


# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
birdy654_cifake_real_and_ai_generated_synthetic_images_path = kagglehub.dataset_download('birdy654/cifake-real-and-ai-generated-synthetic-images')

print('Data source import complete.')

import pandas as pd

#dataset_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/" # For Kaggle notebooks. If you run locally, point this line to the CIFAKE directory
dataset_dir = "/root/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3" # For Kaggle notebooks. If you run locally, point this line to the CIFAKE directory
print("Loading dataset from: " + dataset_dir)


# use https://huggingface.co/docs/datasets/image_load for reference

# Import necessary libraries
image_dict = {}

# Define the list of file names
from pathlib import Path
from tqdm import tqdm
import os
# Initialize empty lists to store file names and labels
file_names = []
labels = []

# Iterate through all image files in the specified directory
for file in sorted((Path('/root/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3').glob('*/*/*.*'))):
    label = str(file).split('/')[-2]  # Extract the label from the file path
    labels.append(label)  # Add the label to the list
    file_names.append(str(file))  # Add the file path to the list

# Print the total number of file names and labels
print(len(file_names), len(labels))

# Create a pandas dataframe from the collected file names and labels
df = pd.DataFrame.from_dict({"image": file_names, "label": labels})
print(df.shape)

# from sklearn.utils import shuffle

# Shuffle the DataFrame
#df_shuffled = shuffle(df)
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

df_train = df_shuffled[:int(0.8*len(df_shuffled))]
df_test = df_shuffled[int(0.8*len(df_shuffled)):]

100%|██████████| 105M/105M [00:00<00:00, 144MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3
Data source import complete.
Loading dataset from: /root/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3
120000 120000
(120000, 2)


In [ ]:
df_test.shape

(24000, 2)

In [ ]:
df_train.to_csv("df_train_clip.csv", index=False)
df_test.to_csv("df_test_clip.csv", index=False)

## Generating CLIP Embeddings

In [ ]:
# Initialize CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)



def generate_embeddings(image):
  # Add tqdm progress bar
  # for entry in tqdm(dataset, desc="Generating embeddings"):
  #     img_path = entry["path"]
  #     label = entry["label"]

  #     Load image
  # image = Image.open(img_path).convert("RGB")

  # Preprocess image
  inputs = processor(images=image, return_tensors="pt").to(device)

  # Get image embeddings
  with torch.no_grad():
      image_embeds = model.get_image_features(**inputs)

  # Normalize embeddings
  image_embeds = image_embeds / image_embeds.norm(p=2, dim=-1, keepdim=True)
  # embeddings.append({
  #     "path": img_path,
  #     "label": label,
  #     "embedding": image_embeds.squeeze(0).cpu().numpy()
  # })

  return image_embeds

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
embeddings_train = []

for i in range(len(df_train)):
  if(i%1000 == 0):
    print(f"On epoch {i}, we have reached {i*100/len(df_train)}% of the way")
  image = Image.open(df_shuffled.loc[i]["image"])
  embed = generate_embeddings(image)
  embeddings_train.append(embed)


embeddings_test = []

for i in range(len(df_test)):
  if(i%1000 == 0):
    print(f"On epoch {i}, we have reached {i*100/len(df_test)}% of the way")
  image = Image.open(df_shuffled.loc[i]["image"])
  embed = generate_embeddings(image)
  embeddings_test.append(embed)

print("embeddings generated successfully")

embeddings generated successfully


## Prepare Data

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/AI/Data/Adobe2024_InterIIT/TrainingCLIP/df_train_clip.csv")
test_df = pd.read_csv("/content/drive/MyDrive/AI/Data/Adobe2024_InterIIT/TrainingCLIP/df_test_clip.csv")
traininglabels_worded = train_df["label"].tolist()
testlabels_worded = test_df["label"].tolist()

traininglabels = []
testlabels = []

for i in range(len(traininglabels_worded)):
  if(traininglabels_worded[i] == "REAL"):
    traininglabels.append(1)
  else:
    traininglabels.append(0)

for i in range(len(testlabels_worded)):
  if(testlabels_worded[i] == "REAL"):
    testlabels.append(1)
  else:
    testlabels.append(0)

#traininglabels contain all the data
# train_embeddings = torch.load("/content/drive/MyDrive/AI/Data/Adobe2024_InterIIT/embeddings_train.pt", map_location=torch.device('cpu'))
# test_embeddings = torch.load("/content/drive/MyDrive/AI/Data/Adobe2024_InterIIT/embeddings_test.pt", map_location=torch.device('cpu'))
train_embeddings = torch.load("/content/drive/MyDrive/AI/Data/Adobe2024_InterIIT/TrainingCLIP/embeddings_train.pt")
test_embeddings = torch.load("/content/drive/MyDrive/AI/Data/Adobe2024_InterIIT/TrainingCLIP/embeddings_test.pt")

<ipython-input-12-682390fa60d2>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_embeddings = torch.load("/content/drive/MyDrive/AI/Data/Adobe2024_InterIIT/TrainingCL

In [ ]:
len(train_embeddings)

96000

In [ ]:
train_embeddings = torch.stack(train_embeddings).to(device)
test_embeddings = torch.stack(test_embeddings).to(device)
traininglabels = torch.tensor(traininglabels).to(device)
testlabels = torch.tensor(testlabels).to(device)


class TrainDS(Dataset):
  def __init__(self):
    self.x = train_embeddings.to(device)
    self.y = traininglabels.to(device)
    self.n_samples = len(train_embeddings)

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.n_samples


class TestDS(Dataset):
  def __init__(self):
    self.x = test_embeddings.to(device)
    self.y = testlabels.to(device)
    self.n_samples = len(test_embeddings)

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.n_samples

train_ds = TrainDS()
test_ds = TestDS()

first_data = test_ds[0]
features, labels = first_data
print(features, labels)


train_dataloader = DataLoader(dataset = train_ds,
                              batch_size = 64,
                              shuffle = True,)

test_dataloader = DataLoader(dataset = test_ds,
                              batch_size = 64,
                              shuffle = False,)

# num_workers = os.cpu_count()
# traininglabels, testlabels
# train_embeddings, test_embeddings

tensor([[-8.7257e-04, -1.9535e-02,  3.3185e-02,  3.4779e-04,  3.5558e-03,
         -2.5704e-02,  5.2675e-02,  1.4164e-02,  1.1723e-02,  3.2627e-02,
          2.0427e-02,  2.2363e-02,  3.4922e-02,  3.3942e-02, -2.8948e-03,
          1.9436e-02,  5.8312e-02,  3.0569e-02,  2.6818e-02, -3.6915e-04,
          7.3715e-03, -2.7406e-02,  3.4366e-02, -1.8977e-02, -3.0820e-02,
         -1.0898e-02, -2.3490e-02,  4.6446e-03,  3.1867e-02, -1.0991e-02,
          2.7208e-02, -4.5901e-02,  2.7496e-02,  5.4858e-03, -4.5776e-02,
         -4.2436e-02,  6.1906e-03, -1.2556e-02, -6.5514e-02,  1.4790e-01,
         -3.0279e-02, -5.1343e-02, -1.6756e-04,  4.7338e-03,  5.4913e-03,
         -6.6648e-02,  8.6520e-03,  3.2008e-02,  4.2624e-02,  2.4663e-03,
          1.5844e-02,  6.5223e-02,  7.8022e-04,  3.6556e-02,  4.0261e-02,
         -4.0480e-03,  3.2004e-02,  5.8649e-03, -5.6866e-02, -3.6633e-02,
          4.5552e-02,  1.3767e-02,  2.8289e-02, -2.0343e-02, -1.3936e-02,
          3.5862e-04, -8.6034e-03,  4.

In [ ]:
len(train_ds)
#test_embeddings.shape

96000

## Neural Network

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # self.fc2 = nn.Linear(512, 256)
        # self.fc4 = nn.Linear(256, 64)
        # self.fc5 = nn.Linear(64,16)
        # self.fc7 = nn.Linear(16,8)
        # self.fc8 = nn.Linear(8,4)
        # self.fc9 = nn.Linear(4,1)
        self.fc1 = nn.Linear(512, 124)
        self.bn1 = nn.BatchNorm1d(124)
        self.fc2 = nn.Linear(124, 32)
        self.fc3 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        #x = x.view(x.size(0), -1)
        # if len(x.shape) == 1:
        #     x = x.unsqueeze(0)
        # x = self.bn1(torch.relu(self.fc1(x)))
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

In [ ]:
model = NeuralNetwork()

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss with Logits
#criterion = nn.MSELoss()  # Mean Squared Error Loss with Logits
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

NeuralNetwork(
  (fc1): Linear(in_features=512, out_features=124, bias=True)
  (bn1): BatchNorm1d(124, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=124, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
def rounder(num):
  distfrom_one = abs(num-1)
  distfrom_zero = abs(num)
  if(distfrom_one < distfrom_zero):
    return 1
  else:
    return 0

def train_accuracy(model):
  accuracy = 0
  model.eval()
  with torch.inference_mode():
    output = model(train_embeddings) # should return something of the size
    for i in range(24000):
      pred = output[i].item()
      pred = rounder(pred)
      if(pred == traininglabels[i]):
        accuracy += 1
  return accuracy*100/24000

def test_accuracy(model):
  accuracy = 0
  model.eval()
  with torch.inference_mode():
    output = model(test_embeddings) # should return something of the size
    for i in range(24000):
      pred = output[i].item()
      pred = rounder(pred)
      if(pred == testlabels[i]):
        accuracy += 1
  return accuracy*100/24000

In [ ]:
for epoch in tqdm(range(num_epochs)):
    print(f"Epoch: {epoch}\n-----")
    running_loss = 0.0

    for batch, (inputs, labels) in enumerate(train_dataloader):
      model.train()
      # inputs, labels = inputs.view(inputs.size(0), -1).to(device), labels.float().to(device)
      inputs, labels = inputs.to(device), labels.view(-1, 1).float().to(device)

      optimizer.zero_grad()

      outputs = model(inputs).reshape(64, 1)
      #loss = criterion(outputs.squeeze(), labels)
      loss = criterion(outputs, labels)
      running_loss += loss.item()

      loss.backward()
      optimizer.step()

      if(batch%400 == 0):
        print(f"Looked at {batch*len(inputs)}/{len(train_dataloader.dataset)} samples.")
        print(f"Accuracy = {train_accuracy(model):.2f}")

    running_loss /= len(train_dataloader)



    #print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
-----
Looked at 0/96000 samples.
Accuracy = 49.14
Looked at 25600/96000 samples.
Accuracy = 50.86
Looked at 51200/96000 samples.
Accuracy = 80.78
Looked at 76800/96000 samples.
Accuracy = 88.00


 20%|██        | 1/5 [00:09<00:39,  9.80s/it]

Epoch: 1
-----
Looked at 0/96000 samples.
Accuracy = 88.59
Looked at 25600/96000 samples.
Accuracy = 90.32
Looked at 51200/96000 samples.
Accuracy = 90.89
Looked at 76800/96000 samples.
Accuracy = 91.08


 40%|████      | 2/5 [00:18<00:26,  8.89s/it]

...

 60%|██████    | 3/5 [00:26<00:17,  8.78s/it]

Epoch: 3
-----
Looked at 0/96000 samples.
Accuracy = 92.28
Looked at 25600/96000 samples.
Accuracy = 92.43
Looked at 51200/96000 samples.
Accuracy = 92.35
Looked at 76800/96000 samples.
Accuracy = 92.50


 80%|████████  | 4/5 [00:35<00:08,  8.92s/it]

Epoch: 4
-----
Looked at 0/96000 samples.
Accuracy = 92.68
Looked at 25600/96000 samples.
Accuracy = 92.83
Looked at 51200/96000 samples.
Accuracy = 92.78
Looked at 76800/96000 samples.
Accuracy = 93.00


100%|██████████| 5/5 [00:43<00:00,  8.72s/it]


#### Accuracy

In [ ]:
# Example output
outputs = []

for i in range(10):
  outputs.append(model(test_embeddings[i]).item())

outputs

[0.999966025352478,
 0.04508163407444954,
 0.04508163407444954,
 0.04508163407444954,
 0.04508163407444954,
 0.9999878406524658,
 0.04508163407444954,
 0.04508163407444954,
 0.04508163407444954,
 0.9999988079071045]

In [ ]:
model.eval()
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.view(inputs.size(0), -1).to(device), labels.float().to(device)
        outputs = model(inputs)

        loss = criterion(outputs.squeeze(), labels)
        val_loss += loss.item()

        predicted = torch.sigmoid(outputs).squeeze() > 0.5
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Validation Loss: {val_loss / len(test_loader):.4f}")
print(f"Validation Accuracy: {correct / total * 100:.2f}%")

#### Save and Load Model

In [ ]:
model_path = Path("Model")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "CLIPNeural_One.pth"

model_save_path = model_path / model_name

print(f"Saving model to: {model_save_path}")
torch.save(obj=model.state_dict(),
           f=model_save_path)

Saving model to: Model/CLIPNeural_One.pth


In [ ]:
loaded_model = NeuralNetwork().to(device)
loaded_model.load_state_dict(torch.load(f="/content/CLIPNeural_One.pth"))

<ipython-input-9-787f74348828>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(f="/content/CLIPNeural_One.pth"))


<All keys matched successfully>

In [ ]:
get_accuracy(loaded_model)

94.20416666666667